In [ ]:
%run Utils/Functions/core

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from pyspark.sql.functions import col
from uuid import uuid4
import time
from datetime import datetime, timedelta, timezone
import concurrent.futures
from time import sleep, mktime
import json

In [ ]:
%sql
use catalog prod;

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

dt_ingestao = getArgument("dt_ingestao").upper().strip()

location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]
location_flat_file = spark.sql("show external locations").select("url").where("name = 'flatfile-area'").collect()[0][0]

In [ ]:
# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
format_date = '%Y-%m-%d'
dt_ingestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dt_ingestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# Transforma as datas em STRING no formato date para usar na API
since = dt_inicio.strftime(format_date)
until = dt_fim.strftime(format_date)

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

##CONFIGURAÇÕES DE PROCESSO DE CARGA

In [ ]:
access_token = '<access_token>'
params = {'access_token': f'{access_token}'}
domain = "graph.facebook.com"
version = "v17.0"
baseUrl = f"https://{domain}/{version}"

threads = 10    

In [ ]:
df_param = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'facebook_ads'
    and (ds_Custom_Field != 'id' or ds_Custom_Field is null)
""")

display(df_param)
data_param = df_param.collect()

In [ ]:
df_param_id = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'facebook_ads' and ds_Custom_Field = 'id'
""")

display(df_param_id)
data_param_id = df_param_id.collect()

## SCRIPT DA API

In [ ]:
def get_tables_from_ids(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing, ids, act_ids):
    try:
        inicio = time.time()

        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
        
        data_format = []
        for id,act_id in zip(ids,act_ids):
            # Declarando as variáveis
            next = None
            limite = 200
            pagina = 1
            error_sleep = 60

            print(f"Tabela {param.ds_Nome_Arquivo_Landing} ---> ID: {id} | {act_id} <--- ACCOUNT ID! Começando carga...")

            # Pega o endpoint que utilizará no request
            # Verificação para casos especiais que salvam a última URL que processou
            end_point_page = baseUrl+param.ds_Url.replace('@accountId', str(id)).replace('@dtInicio', since).replace('@dtFim', until)
            req = requests.get(end_point_page, params=params)
            data = req.json()
            data_list = []
            consumo = dict(req.headers)

            try:
                cpu = consumo['x-business-use-case-usage']
                cpu = json.loads(cpu)
                consumo_cpu = int(cpu[str(act_id)][0]['total_cputime'])
                consumo_time = int(cpu[str(act_id)][0]['total_time'])
                
                if consumo_cpu > 80:
                    print(f'Consumo excessivo de CPU da origem! Esperando um pouco antes de continuar o processamento.\nNível da CPU ---> {consumo_cpu}')
                    sleep(1200)
                elif consumo_time > 80:
                    print(f'Consumo excessivo de tempo da origem! Esperando um pouco antes de continuar o processamento.\nNível total de tempo --> {consumo_time}')
                    sleep(1200)
            except:
                print(f'Tabela {param.nm_Item_Origem} não possui header de consumo!')

            # Verifica se a primeira requisição teve erro
            if req.status_code != 200:
                print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na primeira requisição do ID {id}!")
            else: # Caso a response da requisição seja 200 (sucesso)
                # Armazena o conteúdo da API na variável data em formato JSON
                data = req.json()
                
                end_point_page = None
                data_keys = list(data.keys())

                # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                if param.nm_Definition is not None:
                    campo_data = param.nm_Definition.split(',')
                    if len(campo_data) == 2:
                        data_list.extend(data[campo_data[0]][campo_data[1]])
                    else:
                        data_list.extend(data[campo_data[0]])
                else:
                    data_list.extend(data[param.nm_Item_Origem])

                if 'paging' in data_keys:
                    if 'next' in data['paging'].keys():
                        next = data['paging']['next']
                        end_point_page = next
                    else:
                        next = None

                print(f'Tabela {param.nm_Item_Origem} ---> ID: {id} | {act_id} <--- ACCOUNT ID teve {pagina} páginas adicionadas com êxito!')

            while next is not None:
                req = requests.get(end_point_page, params=params)
                data = req.json()
                pagina += 1
                consumo_cpu = dict(req.headers)

                try:
                    cpu = consumo['x-business-use-case-usage']
                    cpu = json.loads(cpu)
                    consumo_cpu = int(cpu[str(act_id)][0]['total_cputime'])
                    consumo_time = int(cpu[str(act_id)][0]['total_time'])
                                        
                    if consumo_cpu > 80:
                        print(f'Consumo excessivo de CPU da origem! Esperando um pouco antes de continuar o processamento.\nNível da CPU ---> {consumo_cpu}')
                        sleep(1200)
                    elif consumo_time > 80:
                        print(f'Consumo excessivo de tempo da origem! Esperando um pouco antes de continuar o processamento.\nNível total de tempo --> {consumo_time}')
                        sleep(1200)
                except:
                    print(f'Tabela {param.nm_Item_Origem} não possui header de consumo!')
                
                # Verifica se a primeira requisição teve erro
                if req.status_code != 200:
                    print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na página {pagina}!")
                    pagina -= 1
                    print(f'{req.status_code}->{req.content}')
                    sleep(300)
                # Caso a response da requisição seja 200 (sucesso) 
                else:
                    sleep(error_sleep)
                    # Armazena o conteúdo da API na variável data em formato JSON
                    data = req.json()
                    end_point_page = None

                    data_keys = list(data.keys())

                    # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                    if param.nm_Definition is not None:
                        campo_data = param.nm_Definition.split(',')
                        if len(campo_data) == 2:
                            data_list.extend(data[campo_data[0]][campo_data[1]])
                        else:
                            data_list.extend(data[campo_data[0]])
                    else:
                        data_list.extend(data[param.nm_Item_Origem])

                    if 'paging' in data_keys:
                        if 'next' in data['paging'].keys():
                            next = data['paging']['next']
                            end_point_page = next
                        else:
                            next = None
                    else:
                        next = None
                    
                    print(f'Tabela {param.nm_Item_Origem} ---> ID: {id} | {act_id} <--- ACCOUNT ID teve {pagina} páginas adicionadas com êxito!')

            for data in data_list:
                data_format.append(data)

        fim = time.time()
        
        # Subtrai a variável "início" pela "fim" para obter o tempo total de execução da tabela e armazena na váriavel "tempo_exec"
        tempo_exec = fim - inicio

        print("\n"+f"A tabela {param.nm_Item_Origem} teve {pagina} páginas carregadas com êxito!")
        print(f"Tamanho total de registros {len(data_format)}")
        print(f"O tempo de execução da tabela {param.nm_Item_Origem} foi {tempo_exec}")

        # PROCESSO DE CARGA DOS DADOS NO BLOB
        fn_SaveJson(data_format, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())

        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)
    except Exception as error:
        msg = f"Erro na tabela: {param.ds_Nome_Arquivo_Landing} ---> ID: {id}. ERROR: {error}"
        print(msg)
        return msg

In [ ]:
def get_tables(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing):
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dtIngestao)
        if param.vl_Ultimo_Incremento is not None:
            vlUltimoIncremento = datetime.strptime(param.vl_Ultimo_Incremento, format_timestamp)
            vlUltimoIncremento = int(mktime(vlUltimoIncremento.timetuple()))
        else:
            vlUltimoIncremento = dt_inicio_unix

        vlUltimoIncremento = str(vlUltimoIncremento)

        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()

        inicio = time.time()

        data_format = []
        pagina = 1

        for act_id in param.nm_Campo_Fields.split(','):
            # Declarando as variáveis
            next = None
            limite = 200
            data_list = []

            print(f"Tabela {param.ds_Nome_Arquivo_Landing}! Começando carga...")

            # Pega o endpoint que utilizará no request
            # Verificação para casos especiais que salvam a última URL que processou
            if param.ds_Custom_Field is None:
                end_point_page = f"{baseUrl}{param.ds_Url.replace('@accountId', f'act_{act_id}').replace('@dtInicio', since).replace('@dtFim', until)}"
            else:
                # end_point_page = baseUrl+param.ds_Custom_Field
                end_point_page = f"{baseUrl}{param.ds_Url.replace('@accountId', f'act_{act_id}').replace('@dtInicio', since).replace('@dtFim', until)}"
            
            req = requests.get(end_point_page, params=params)

            # Verifica se a primeira requisição teve erro
            if req.status_code != 200:
                print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na primeira requisição!")
                print(req.content)
                sleep(300)
            # Caso a response da requisição seja 200 (sucesso) 
            else:
                # Armazena o conteúdo da API na variável data em formato JSON
                data = req.json()
                consumo_cpu = dict(req.headers)

                try:
                    res = consumo_cpu['x-business-use-case-usage']
                    res = json.loads(res)
                    consumo_cpu = int(res[act_id][0]['total_cputime'])
                    
                    if consumo_cpu > 80:
                        print(f'Consumo excessivo de CPU da origem! Esperando um pouco antes de continuar o processamento.\nNível da CPU ---> {consumo_cpu}')
                        sleep(600)
                except:
                    print(f'Tabela {param.nm_Item_Origem} não possui header de consumo!')

                end_point_page = None
                data_keys = list(data.keys())

                # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                if param.nm_Definition is not None:
                    campo_data = param.nm_Definition.split(',')
                    if len(campo_data) == 2:
                        data_list.extend(data[campo_data[0]][campo_data[1]])
                    else:
                        data_list.extend(data[campo_data[0]])
                else:
                    data_list.extend(data[param.nm_Item_Origem])
       
                if 'paging' in data_keys:
                    if 'next' in data['paging'].keys():
                        next = data['paging']['next']
                        end_point_page = next
                    else:
                        next = None

            while next is not None:
                print(f'Começando carga página {pagina+1} da tabela {param.nm_Item_Origem}')
                req = requests.get(end_point_page, params=params)
                data = req.json()
                pagina += 1
                
                # Verifica se há erro na tabela Ads, caso sim, carrega as páginas já processadas
                if param.nm_Item_Origem == 'ads' and req.status_code == 400:
                    erro = req.content
                    print("\n"+f'Tabela {param.ds_Nome_Arquivo_Landing} ---> {id}, com erro: {erro}. Concluindo carga com dados carregados!')
                    break
                elif param.nm_Item_Origem == 'ad_creatives' and req.status_code != 200:
                    print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na página {pagina}! Alterando os limites!")
                    pagina -= 1
                    print(f'{req.status_code}->{req.content}')
                    end_point_page = end_point_page.replace(f'limit={limite}', f'limit={limite-50}')
                    limite = limite - 50
                    sleep(60)
                # Verifica se a primeira requisição teve erro
                elif req.status_code != 200:
                    print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na página {pagina}!")
                    pagina -= 1
                    print(f'{req.status_code}->{req.content}')
                    sleep(300)

                # Caso a response da requisição seja 200 (sucesso) 
                else:
                    # Armazena o conteúdo da API na variável data em formato JSON
                    data = req.json()
                    consumo_cpu = dict(req.headers)

                    try:
                        res = consumo_cpu['x-business-use-case-usage']
                        res = json.loads(res)
                        consumo_cpu = int(res[act_id][0]['total_cputime'])
                        
                        if consumo_cpu > 80:
                            print(f'Consumo excessivo de CPU da origem! Esperando um pouco antes de continuar o processamento.\nNível da CPU ---> {consumo_cpu}')
                            sleep(600)
                    except:
                        print(f'Tabela {param.nm_Item_Origem} não possui header de consumo!')

                    end_point_page = None

                    data_keys = list(data.keys())

                    # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                    if param.nm_Definition is not None:
                        campo_data = param.nm_Definition.split(',')
                        if len(campo_data) == 2:
                            data_list.extend(data[campo_data[0]][campo_data[1]])
                        else:
                            data_list.extend(data[campo_data[0]])
                    else:
                        data_list.extend(data[param.nm_Item_Origem])

                    if 'paging' in data_keys:
                        if 'next' in data['paging'].keys():
                            next = data['paging']['next']
                            end_point_page = next
                        else:
                            next = None
                    else:
                        next = None

            for data in data_list:
                data_format.append(data)

        fim = time.time()
        
        # Subtrai a variável "início" pela "fim" para obter o tempo total de execução da tabela e armazena na váriavel "tempo_exec"
        tempo_exec = fim - inicio

        print("\n"+f"A tabela {param.nm_Item_Origem} teve {pagina} páginas carregadas com êxito!")
        print(f"Tamanho total de registros {len(data_format)}")
        print(f"O tempo de execução da tabela {param.nm_Item_Origem} foi {tempo_exec}")

        # PROCESSO DE CARGA DOS DADOS NO BLOB
        fn_SaveJson(data_format, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())

        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)

        if param.nm_Item_Origem in ['ad_campaigns']:
            list_ids = []
            list_act_ids = []
            subtables_campaigns = ['ad_insights','ad_insights_age_and_gender','ad_insights_country','ad_insights_dma','ad_insights_hourly_stats_by_audience_timezone','ad_insights_platform_device','ad_insights_region']

            data_param_id = df_param_id.filter(col('nm_Item_Origem').isin(subtables_campaigns)).collect()
          
            for data in data_format:
                list_ids.append(data['id'])
                list_act_ids.append(data['account_id'])

            for row in data_param_id:
                # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
                subtask = executor.submit(get_tables_from_ids, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing, list_ids, list_act_ids))
                # Adiciona a tarefa à lista de tarefas
                subtasks.append(subtask)

    except Exception as error:
        msg = f"Erro na tabela: {param.ds_Nome_Arquivo_Landing}. ERROR: {error}"
        print(msg)
        return msg

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []
subtasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
executor = concurrent.futures.ThreadPoolExecutor(max_workers=threads)

# Percorre todas as pastas do diretório de origem
for row in data_param:
    # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
    task = executor.submit(get_tables, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing))
    # Adiciona a tarefa à lista de tarefas
    tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

executor.shutdown()

In [ ]:
tarefas = tasks + subtasks

for task in tarefas:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)
        pass